In [9]:
import pandas as pd
import geopandas as gpd

In [17]:
#pull our pdew data and the geospatial dataframe pulled from the PDEW_eda script with city locations
all_pdew = pd.read_csv('C:\\Users\\olubl\\OneDrive\\14th grade\\PDEW_Predict\\data\\pdew_3q_2024.csv')
all_pdew = all_pdew[['city1','city2','nsmiles','passengers','fare','carrier_lg']]
all_cities_geo = gpd.read_file('C:\\Users\\olubl\\OneDrive\\14th grade\\PDEW_Predict\\city_coords.geojson')

In [18]:
all_pdew.head()

,city1,city2,nsmiles,passengers,fare,carrier_lg
0,"Allentown/Bethlehem/Easton, PA","Dallas/Fort Worth, TX",1307,18,367.69,AA
1,"Allentown/Bethlehem/Easton, PA","Denver, CO",1539,89,138.60,G4
2,"Allentown/Bethlehem/Easton, PA","Atlanta, GA (Metropolitan Area)",692,107,319.69,DL
3,"Allentown/Bethlehem/Easton, PA","Austin, TX",1441,13,256.48,AA
4,"Allentown/Bethlehem/Easton, PA","Nashville, TN",685,46,130.88,G4


In [20]:
#descriptive stats for each numeric column
all_pdew.describe()

,nsmiles,passengers,fare
count,6611.000000,6611.000000,6611.000000
mean,1091.091514,203.439873,282.530593
std,584.066684,688.511938,79.852923
min,70.000000,10.000000,56.470000
25%,637.000000,18.000000,239.730000
50%,964.000000,36.000000,281.650000
75%,1471.000000,112.000000,327.310000
max,2783.000000,16339.000000,705.010000


In [19]:
#now we can get additional columns "city1_geo" and "city2_geo" showing the rough geometry of each citymarket

In [8]:
#descriptive stats for each numeric column
all_pdew.describe()

,nsmiles,passengers,fare
count,6611.000000,6611.000000,6611.000000
mean,1091.091514,203.439873,282.530593
std,584.066684,688.511938,79.852923
min,70.000000,10.000000,56.470000
25%,637.000000,18.000000,239.730000
50%,964.000000,36.000000,281.650000
75%,1471.000000,112.000000,327.310000
max,2783.000000,16339.000000,705.010000


In [ ]:
#as one of the features of our model will be catchment population for each citymarket, we need to pull this data.

In [ ]:
#feature engineering with population gravity (log(city1 * city2))

In [ ]:
#baseline regression - gravity based